In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
data_path_and_labels = pd.read_csv("../mapped_data/camel-1.2.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [6]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [7]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent'

In [8]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [9]:
d = pd.read_csv("../mapped_data/camel-1.2.csv")
d['data_path'][0]

'dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [10]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [11]:
d

,data_path,bugs
0,../dataset/promise/defect data/camel-1.2/src/c...,0
1,../dataset/promise/defect data/camel-1.2/src/c...,0
2,../dataset/promise/defect data/camel-1.2/src/c...,0
3,../dataset/promise/defect data/camel-1.2/src/c...,0
4,../dataset/promise/defect data/camel-1.2/src/c...,0
...,...,...
590,../dataset/promise/defect data/camel-1.2/src/c...,1
591,../dataset/promise/defect data/camel-1.2/src/c...,0
592,../dataset/promise/defect data/camel-1.2/src/c...,1
593,../dataset/promise/defect data/camel-1.2/src/c...,0


In [12]:
train, test = train_test_split(d, test_size=0.2)

In [13]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [14]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [15]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

6319.0

In [16]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [17]:
train_y

array([ 2,  0,  0,  2,  1,  1,  5,  0,  0,  0,  4,  4,  0,  0,  0,  1,  2,
       10,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  3,  1,  2,  0,  0,  1,
        2,  0,  0,  1,  0,  4,  1,  0,  0,  1,  0,  0,  0,  0,  2,  0,  2,
        0,  0,  3,  1,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  2,  1,  2,  0,  0,  0,  0,  3,  0,  0,  0,  2,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  2,  9,  1,  0,  0,  0,  0,  0,  1,
        0,  4,  0,  0,  0,  1,  0,  2,  0,  2,  3,  1,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  2,  0,  3,  0,  0,  1,  0,  0,  0,  1,
        0,  0,  2,  0,  0,  0,  7,  1,  0,  0,  1,  1,  1,  0,  0,  0,  0,
        1,  0,  0,  0,  6,  1,  0,  0,  0,  0,  0,  0,  2,  0,  2,  0,  3,
        0,  0,  0,  0,  2,  4,  0,  0,  0,  5,  0,  0,  2,  2,  1,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  5,  0,  0,  1,  0,  3,  0,  2,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  2,  0,  0,  1,  0,  0,  1,  1,  4,
        4,  1,  3,  0,  4

In [18]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [19]:
train_x.shape

(476, 1765)

In [20]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
15/15 [==============================] - 0s 9ms/step - loss: 4.1354 - mse: 4.1354 - mae: 1.0368 - root_mean_squared_error: 2.0336 - mean_squared_logarithmic_error: 0.4207
Epoch 2/100
15/15 [==============================] - 0s 8ms/step - loss: 3.9070 - mse: 3.9070 - mae: 1.1189 - root_mean_squared_error: 1.9766 - mean_squared_logarithmic_error: 0.4204
Epoch 3/100
15/15 [==============================] - 0s 9ms/step - loss: 3.8534 - mse: 3.8534 - mae: 1.0493 - root_mean_squared_error: 1.9630 - mean_squared_logarithmic_error: 0.3636
Epoch 4/100
15/15 [==============================] - 0s 8ms/step - loss: 3.7795 - mse: 3.7795 - mae: 1.0962 - root_mean_squared_error: 1.9441 - mean_squared_logarithmic_error: 0.3980
Epoch 5/100
15/15 [==============================] - 0s 9ms/step - loss: 3.7006 - mse: 3.7006 - mae: 1.0692 - root_mean_squared_error: 1.9237 - mean_squared_logarithmic_error: 0.3703
Epoch 6/100
15/15 [==============================] - 0s 9ms/step - loss: 3.6421 - mse

15/15 [==============================] - 0s 10ms/step - loss: 1.2217 - mse: 1.2217 - mae: 0.6636 - root_mean_squared_error: 1.1053 - mean_squared_logarithmic_error: 0.1724
Epoch 45/100
15/15 [==============================] - 0s 9ms/step - loss: 1.1817 - mse: 1.1817 - mae: 0.7176 - root_mean_squared_error: 1.0871 - mean_squared_logarithmic_error: 0.1991
Epoch 46/100
15/15 [==============================] - 0s 8ms/step - loss: 1.1388 - mse: 1.1388 - mae: 0.6504 - root_mean_squared_error: 1.0671 - mean_squared_logarithmic_error: 0.1695
Epoch 47/100
15/15 [==============================] - 0s 9ms/step - loss: 1.0710 - mse: 1.0710 - mae: 0.6450 - root_mean_squared_error: 1.0349 - mean_squared_logarithmic_error: 0.1649
Epoch 48/100
15/15 [==============================] - 0s 9ms/step - loss: 1.0321 - mse: 1.0321 - mae: 0.6206 - root_mean_squared_error: 1.0159 - mean_squared_logarithmic_error: 0.1559
Epoch 49/100
15/15 [==============================] - 0s 12ms/step - loss: 0.9894 - mse: 0.9

15/15 [==============================] - 0s 9ms/step - loss: 0.2608 - mse: 0.2608 - mae: 0.2901 - root_mean_squared_error: 0.5107 - mean_squared_logarithmic_error: 0.0527
Epoch 89/100
15/15 [==============================] - 0s 8ms/step - loss: 0.2554 - mse: 0.2554 - mae: 0.2636 - root_mean_squared_error: 0.5054 - mean_squared_logarithmic_error: 0.0463
Epoch 90/100
15/15 [==============================] - 0s 11ms/step - loss: 0.2492 - mse: 0.2492 - mae: 0.2927 - root_mean_squared_error: 0.4992 - mean_squared_logarithmic_error: 0.0534 0s - loss: 0.2449 - mse: 0.2449 - mae: 0.2876 - root_mean_squared_error: 0.4949 - mean_squared_logarithmic_error: 0.05
Epoch 91/100
15/15 [==============================] - 0s 10ms/step - loss: 0.2417 - mse: 0.2417 - mae: 0.2687 - root_mean_squared_error: 0.4916 - mean_squared_logarithmic_error: 0.0471
Epoch 92/100
15/15 [==============================] - 0s 10ms/step - loss: 0.2323 - mse: 0.2323 - mae: 0.2656 - root_mean_squared_error: 0.4820 - mean_squar

In [22]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:29:55 ago. (Use '!kill 21467' to kill it.)

In [23]:
score = model.evaluate(test_x, test_y)
score

4/4 [==============================] - 0s 2ms/step - loss: 1.6047 - mse: 1.6047 - mae: 0.8789 - root_mean_squared_error: 1.2668 - mean_squared_logarithmic_error: 0.2544


[1.604737401008606,
 1.604737401008606,
 0.8789141774177551,
 1.266782283782959,
 0.2544384300708771]

In [24]:
data_path_and_labels = pd.read_csv("../mapped_data/camel-1.2.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [25]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.2/src/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [26]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [27]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764
0,797.0,791.0,2104.0,797.0,2414.0,201.0,4842.0,559.0,2104.0,791.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,799.0,1577.0,182.0,1577.0,934.0,906.0,3561.0,799.0,724.0,2370.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,793.0,1577.0,5938.0,1577.0,3970.0,1577.0,4840.0,1577.0,4615.0,1577.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,798.0,1577.0,4493.0,1577.0,5938.0,728.0,6220.0,1577.0,6277.0,1577.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,791.0,3068.0,1123.0,2989.0,797.0,4474.0,798.0,1577.0,6158.0,3593.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,1303.0,5.0,1577.0,255.0,420.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
591,1303.0,580.0,958.0,1149.0,1577.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
592,570.0,566.0,553.0,3054.0,553.0,2989.0,570.0,3054.0,566.0,553.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,737.0,908.0,4308.0,1577.0,3054.0,737.0,4308.0,908.0,3054.0,1577.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764
0,0,797.0,791.0,2104.0,797.0,2414.0,201.0,4842.0,559.0,2104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,799.0,1577.0,182.0,1577.0,934.0,906.0,3561.0,799.0,724.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,793.0,1577.0,5938.0,1577.0,3970.0,1577.0,4840.0,1577.0,4615.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,798.0,1577.0,4493.0,1577.0,5938.0,728.0,6220.0,1577.0,6277.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,791.0,3068.0,1123.0,2989.0,797.0,4474.0,798.0,1577.0,6158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,1,1303.0,5.0,1577.0,255.0,420.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
591,0,1303.0,580.0,958.0,1149.0,1577.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
592,1,570.0,566.0,553.0,3054.0,553.0,2989.0,570.0,3054.0,566.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,0,737.0,908.0,4308.0,1577.0,3054.0,737.0,4308.0,908.0,3054.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
train, test = train_test_split(data, test_size=0.2)

In [30]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764
532,6,602.0,934.0,906.0,3561.0,602.0,603.0,2896.0,2129.0,5090.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
535,0,604.0,599.0,3109.0,604.0,2181.0,201.0,4842.0,559.0,3109.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
476,1,576.0,4440.0,2129.0,2989.0,495.0,1123.0,6234.0,2958.0,2989.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,0,340.0,3067.0,1123.0,2989.0,351.0,4474.0,984.0,5249.0,3263.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0,1612.0,413.0,6049.0,413.0,6042.0,4222.0,2129.0,5249.0,6049.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,5,568.0,1596.0,5236.0,201.0,2181.0,2867.0,1596.0,6022.0,1577.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,0,967.0,984.0,4416.0,967.0,967.0,4474.0,984.0,4416.0,4474.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,0,1309.0,938.0,906.0,3542.0,1309.0,1309.0,1275.0,3562.0,938.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
584,4,1107.0,1577.0,3988.0,1577.0,4003.0,1577.0,4002.0,1577.0,4306.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764
298,0,1584.0,1592.0,2141.0,1592.0,1572.0,5236.0,1572.0,2141.0,3762.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,0,1227.0,1225.0,4989.0,1227.0,2912.0,1577.0,2414.0,201.0,4989.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,5,52.0,1577.0,957.0,934.0,906.0,3561.0,52.0,1351.0,5223.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,0,342.0,2129.0,4172.0,342.0,5556.0,2884.0,2129.0,3993.0,2884.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133,0,351.0,340.0,2104.0,553.0,2989.0,351.0,2414.0,201.0,2104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0,1084.0,975.0,1577.0,1577.0,4573.0,666.0,1577.0,1577.0,4570.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
285,3,567.0,567.0,3953.0,553.0,566.0,495.0,3957.0,1577.0,5249.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
557,1,477.0,478.0,478.0,2896.0,516.0,2896.0,4961.0,2989.0,553.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,3,759.0,1191.0,2460.0,759.0,2460.0,1191.0,2460.0,2460.0,4961.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [33]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [34]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [35]:
X_train = train.values
X_test = test.values

In [36]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [37]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [38]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
15/15 [==============================] - 1s 4ms/step - loss: 4.5048 - mse: 4.5048 - mae: 0.9888 - root_mean_squared_error: 2.1225 - mean_squared_logarithmic_error: 0.4600
Epoch 2/100
15/15 [==============================] - 0s 4ms/step - loss: 4.3198 - mse: 4.3198 - mae: 1.1356 - root_mean_squared_error: 2.0784 - mean_squared_logarithmic_error: 0.4812
Epoch 3/100
15/15 [==============================] - 0s 4ms/step - loss: 4.1156 - mse: 4.1156 - mae: 1.0013 - root_mean_squared_error: 2.0287 - mean_squared_logarithmic_error: 0.4140
Epoch 4/100
15/15 [==============================] - 0s 3ms/step - loss: 3.9025 - mse: 3.9025 - mae: 1.0706 - root_mean_squared_error: 1.9755 - mean_squared_logarithmic_error: 0.4412
Epoch 5/100
15/15 [==============================] - 0s 3ms/step - loss: 3.9849 - mse: 3.9849 - mae: 1.0209 - root_mean_squared_error: 1.9962 - mean_squared_logarithmic_error: 0.4341
Epoch 6/100
15/15 [==============================] - 0s 3ms/step - loss: 3.8749 - mse

15/15 [==============================] - 0s 3ms/step - loss: 3.4173 - mse: 3.4173 - mae: 1.0600 - root_mean_squared_error: 1.8486 - mean_squared_logarithmic_error: 0.3881
Epoch 46/100
15/15 [==============================] - 0s 3ms/step - loss: 3.3502 - mse: 3.3502 - mae: 1.1019 - root_mean_squared_error: 1.8303 - mean_squared_logarithmic_error: 0.4072
Epoch 47/100
15/15 [==============================] - 0s 3ms/step - loss: 3.2935 - mse: 3.2935 - mae: 1.0462 - root_mean_squared_error: 1.8148 - mean_squared_logarithmic_error: 0.3775
Epoch 48/100
15/15 [==============================] - 0s 3ms/step - loss: 3.3137 - mse: 3.3137 - mae: 1.1013 - root_mean_squared_error: 1.8203 - mean_squared_logarithmic_error: 0.4007
Epoch 49/100
15/15 [==============================] - 0s 3ms/step - loss: 3.2958 - mse: 3.2958 - mae: 1.0460 - root_mean_squared_error: 1.8154 - mean_squared_logarithmic_error: 0.3681
Epoch 50/100
15/15 [==============================] - 0s 3ms/step - loss: 3.2088 - mse: 3.208

15/15 [==============================] - 0s 3ms/step - loss: 3.4236 - mse: 3.4236 - mae: 1.0646 - root_mean_squared_error: 1.8503 - mean_squared_logarithmic_error: 0.3691
Epoch 90/100
15/15 [==============================] - 0s 3ms/step - loss: 3.5863 - mse: 3.5863 - mae: 1.1324 - root_mean_squared_error: 1.8937 - mean_squared_logarithmic_error: 0.4099
Epoch 91/100
15/15 [==============================] - 0s 4ms/step - loss: 3.2138 - mse: 3.2138 - mae: 1.0534 - root_mean_squared_error: 1.7927 - mean_squared_logarithmic_error: 0.3593
Epoch 92/100
15/15 [==============================] - 0s 3ms/step - loss: 3.2510 - mse: 3.2510 - mae: 1.1052 - root_mean_squared_error: 1.8031 - mean_squared_logarithmic_error: 0.3934
Epoch 93/100
15/15 [==============================] - 0s 3ms/step - loss: 3.0448 - mse: 3.0448 - mae: 1.0506 - root_mean_squared_error: 1.7449 - mean_squared_logarithmic_error: 0.3592
Epoch 94/100
15/15 [==============================] - 0s 3ms/step - loss: 2.9325 - mse: 2.932

In [39]:
score = model.evaluate(test_x, test_y)
score

4/4 [==============================] - 0s 2ms/step - loss: 1.6749 - mse: 1.6749 - mae: 0.8608 - root_mean_squared_error: 1.2942 - mean_squared_logarithmic_error: 0.2921


[1.6749351024627686,
 1.6749351024627686,
 0.8608003258705139,
 1.2941927909851074,
 0.2921186089515686]